In [1]:
import boto3
import psycopg2
from sensitive import hidedata

### getting sensitive data

In [3]:
REGION = hidedata.get('region')
ACCESS_KEY = hidedata.get('accesskey')
SECREAT_KEY =hidedata.get('secreatkey')
BUCKET = hidedata.get('bucket')
DATABASE_USER = hidedata.get('user')
DATABASE_PASSWORD = hidedata.get('password')
DATABASE_PORT = hidedata.get('port')
DATABASE_HOST = hidedata.get('host')

In [3]:
# connecting aws to boto3 package using access key and secreat key

s3 = boto3.resource(
    service_name='s3',
    region_name=REGION,
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECREAT_KEY,
)
s3

s3.ServiceResource()

In [4]:
# print bucket names

for bucket in s3.buckets.all():
    print(bucket.name)

c360bucket
codepipeline-us-east-1-678179654666


### Store the bucket files

In [5]:
def files():
    
    # Empty list
    file_names=[]
    
    #selecting bucket
    my_bucket=s3.Bucket(name=BUCKET)

    for file in my_bucket.objects.all():
        # store the files inside the empty list
        file_names.append(file.key)
        
    print(file_names)
    
files()

['sain_lat_long.csv', 'samsung_location.csv']


### Downloading files inside local machine

In [98]:
# The files store inside of the jupyter notebook file(execute "pwd" it shows files path)

# Method 1 for downloading files

def files_download(bucket,access_key,secret_key):
    
    # Create Session
    # Session is an object to create a connection to AWS Service and manage the state of the connection
    session = boto3.Session(
        aws_access_key_id=access_key,
        aws_secret_access_key=secret_key, 
    )

    # Initiate S3 Resource
    s3 = session.resource('s3')

    # Select Your S3 Bucket
    your_bucket = s3.Bucket(bucket)

    # Iterate All Objects in Your S3 Bucket Over the for Loop
    for s3_object in your_bucket.objects.all():

        # Use this statement if your files are available directly in your bucket. 
        your_bucket.download_file(s3_object.key, str(s3_object.key))
        
files_download(BUCKET,ACCESS_KEY,SECREAT_KEY)

In [46]:
# Method 2 for downloading files

def download_s3files(bucketname,access_key,security_key):
    
    # It provides methods to connect with AWS services similar to the AWS API service.
    s3 = boto3.resource('s3',
          aws_access_key_id = access_key,
          aws_secret_access_key =security_key)
    
    # select the bucket
    bucket = s3.Bucket(bucketname)
    
    # Iterate All Objects in Your S3 Bucket Over the for Loop
    for obj in bucket.objects.all():
        filename = obj.key.rsplit('/')[-1]
        bucket.download_file(obj.key, "" + filename)
        
        
download_s3files(BUCKET,ACCESS_KEY,SECREAT_KEY)

### Postgresql

In [122]:
# Createing tables inside posgresql database

def creating_tables():
    
    commands = (
        '''
        CREATE TABLE samsung_location (
            event_type VARCHAR(255),
            imei VARCHAR(255),
            arrival_timestamp VARCHAR(255),
            gpslatitute VARCHAR(255),
            gpaslongitude VARCHAR(255),
            make VARCHAR(255),
            model VARCHAR(255),
            name VARCHAR(255),
            version VARCHAR(255),
            flag VARCHAR(255),
            city VARCHAR(255),
            state VARCHAR(255)
        )
        ''',
        ''' CREATE TABLE sain_lat_long (
                city VARCHAR(255),
                state VARCHAR(255),
                place VARCHAR(255),
                countrycode VARCHAR(255),
                lat VARCHAR(255),
                long VARCHAR(255),
                subarea VARCHAR(255)
                )
        ''', )
    conn = None
    
    try:
        # connect to the PostgreSQL server
        conn = psycopg2.connect(database=DATABASE_USER, user=DATABASE_USER, password=DATABASE_PASSWORD,host=DATABASE_HOST)
        cur = conn.cursor()
        
        # create table one by one
        for command in commands:
            cur.execute(command)
            
            
        # commit the changes
        conn.commit()
            
        # close communication with the PostgreSQL database server
        cur.close() 
        
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        
    finally:
        if conn is not None:
            conn.close()

creating_tables()

### Dumping data from postgresql

In [129]:
def dumping(csv_file_name1,csv_file_name2):
    
    # Connecting postgresql
    conn = psycopg2.connect(database=DATABASE_USER, user=DATABASE_USER, password=DATABASE_PASSWORD,host=DATABASE_HOST)
    
    # Cursor is used to decreases the time for processing set of data
    cur = conn.cursor()
    
    # Queries to create tables
    sql1 = "COPY samsung_location FROM STDIN DELIMITER ',' CSV HEADER"
    sql2 = "COPY sain_lat_long FROM STDIN DELIMITER ',' CSV HEADER"
    
    # Copying Two csv files data
    cur.copy_expert(sql1, open(csv_file_name1, "r"))
    cur.copy_expert(sql2, open(csv_file_name2, "r"))
    
    #Commit Changes
    conn.commit()
    
    #Close connection
    conn.close()
    

    
dumping(r'c:\Users\Venkata Bharadwaj\Documents\Python  task\samsung_location.csv',
        r'c:\Users\Venkata Bharadwaj\Documents\Python  task\sain_lat_long.csv')

### Inserting rows in postgresql table

In [4]:
# Inserting rows inside posgresql database table

def Inserting_rows():
    
    commands = (
        '''
        INSERT INTO samsung_location(event_type,imei,arrival_timestamp,gpslatitute,gpaslongitude,make,model,name,version,flag) values('AB_LOCATION_EVENT',
        's-A0RNt7QCrFminqngkw',
        '2021-09-23 1:43:04 PM',
        '17.38405',
        '78.45636',
        'samsung',
        'SM-A825G',
        'ANDRIOD',
        '7.2',
        'new'
        )
        
        ''',
        '''
        Insert INTO sain_lat_long values('Hyderabad',
        'Telengana',
        'Adilabad',
        'IN',
        '17.38405', 
        '78.45636',
        'Adilabad'
                )
        ''', )
    conn = None
    
    try:
        # connect to the PostgreSQL server
        conn = psycopg2.connect(database=DATABASE_USER, user=DATABASE_USER, password=DATABASE_PASSWORD,host=DATABASE_HOST)
        cur = conn.cursor()
        
        # create table one by one
        for command in commands:
            cur.execute(command)
            
            
        # commit the changes
        conn.commit()
            
        # close communication with the PostgreSQL database server
        cur.close() 
        
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        
    finally:
        if conn is not None:
            conn.close()

Inserting_rows()

### Update the particular row

In [29]:
# Updating row by using psycopg2

def updating_row():
    
    command = '''
    
    UPDATE samsung_location
    
    SET city = sain_lat_long.city,
    state = sain_lat_long.state

    FROM sain_lat_long

    WHERE samsung_location.gpslatitute = sain_lat_long.lat AND samsung_location.gpaslongitude = sain_lat_long.long
    
    '''
       
    conn = None
    
    try:
        # connect to the PostgreSQL server
        conn = psycopg2.connect(database=DATABASE_USER, user=DATABASE_USER, password=DATABASE_PASSWORD,host=DATABASE_HOST)
        cur = conn.cursor()
        
        cur.execute(command)
            
        # commit the changes
        conn.commit()
            
        # close communication with the PostgreSQL database server
        cur.close() 
        
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        
    finally:
        if conn is not None:
            conn.close()
            
            
updating_row()